In [5]:

import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier

codes are from here:
https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652

In [6]:

dataset = pd.read_csv('data/data.tsv', sep='\t')

CMStatus = np.array(dataset.loc[:, dataset.columns == 'cancer'])

otus= np.array(dataset.loc[: , dataset.columns != 'cancer'])

# print (CMStatus)

In [7]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_CMStatus, test_CMStatus,train_otus, test_otus  = train_test_split(CMStatus, otus, test_size = 0.25, random_state = 42)

In [8]:
print('Training Features Shape:', train_otus.shape)
print('Training Labels Shape:', train_CMStatus.shape)
print('Testing Features Shape:', test_otus.shape)
print('Testing Labels Shape:', test_CMStatus.shape)

Training Features Shape: (219, 5206)
Training Labels Shape: (219, 1)
Testing Features Shape: (73, 5206)
Testing Labels Shape: (73, 1)


In [9]:
from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()

rfc.fit(train_otus, train_CMStatus.ravel())

rfc_predict = rfc.predict(test_otus)



In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

rfc_cv_score = cross_val_score(rfc, otus, CMStatus.ravel(), cv=10, scoring='roc_auc')

In [11]:
print("=== Confusion Matrix ===")
print(confusion_matrix(test_CMStatus, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(test_CMStatus, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[43  5]
 [12 13]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.78      0.90      0.83        48
           1       0.72      0.52      0.60        25

    accuracy                           0.77        73
   macro avg       0.75      0.71      0.72        73
weighted avg       0.76      0.77      0.76        73



=== All AUC Scores ===
[0.7662037  0.84027778 0.65196078 0.77205882 0.74019608 0.4877451
 0.79656863 0.95833333 0.64215686 0.74754902]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7403050108932463


In [12]:
from sklearn.model_selection import RandomizedSearchCV
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the model
rfc_random.fit(train_otus, train_CMStatus.ravel())
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END .max_depth=260, max_features=sqrt, n_estimators=800; total time=   3.7s
[CV] END .max_depth=260, max_features=sqrt, n_estimators=800; total time=   3.7s
[CV] END max_depth=140, max_features=auto, n_estimators=1000; total time=   4.4s
[CV] END max_depth=140, max_features=auto, n_estimators=1000; total time=   4.5s[CV] END max_depth=140, max_features=auto, n_estimators=1000; total time=   4.5s

[CV] END max_depth=100, max_features=auto, n_estimators=1400; total time=   6.1s
[CV] END max_depth=100, max_features=auto, n_estimators=1400; total time=   6.1s
[CV] END max_depth=100, max_features=auto, n_estimators=1400; total time=   6.2s
[CV] END .max_depth=260, max_features=sqrt, n_estimators=800; total time=   3.3s
[CV] END max_depth=300, max_features=auto, n_estimators=1000; total time=   4.1s
[CV] END .max_depth=420, max_features=sqrt, n_estimators=600; total time=   2.5s
[CV] END .max_depth=420, max_features=sqrt, n_

In [13]:
rfc = RandomForestClassifier(n_estimators=200, max_depth=220, max_features='auto')
rfc.fit(train_otus,train_CMStatus.ravel())
rfc_predict = rfc.predict(test_otus)
rfc_cv_score = cross_val_score(rfc, otus, CMStatus.ravel(), cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(test_CMStatus.ravel(), rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(test_CMStatus.ravel(), rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[47  1]
 [13 12]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.78      0.98      0.87        48
           1       0.92      0.48      0.63        25

    accuracy                           0.81        73
   macro avg       0.85      0.73      0.75        73
weighted avg       0.83      0.81      0.79        73



=== All AUC Scores ===
[0.83101852 0.88194444 0.66176471 0.76470588 0.74754902 0.62745098
 0.75       0.91911765 0.66176471 0.75      ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7595315904139434


In [15]:
test= pd.read_csv('data/predata.tsv', sep='\t')

testotus= np.array(test.loc[: , test.columns != 'cancer'])

print(rfc.predict(testotus))




[0 0 1 1 1 0 1]
